<a href="https://colab.research.google.com/github/crim-ca/geoimagenet/blob/master/classif_model_packaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Packaging for GeoImageNet

This notebook explains how to package a ResNet model using the thelper library. More details about thelper export function can be found here:

https://thelper.readthedocs.io/en/latest/user-guide.html#export-model

First, we need to install the required libraries.

### Environnement Configuration

In [1]:
%%bash
pip3 --quiet install  torch torchvision pillow gitpython lz4 matplotlib numpy pyyaml scikit-learn six tqdm h5py opencv-python  pretrainedmodels albumentations pyyaml
pip3 --quiet install affine geojson shapely pyproj hdf5plugin
pip uninstall thelper
rm -rf ./thelper
git clone https://github.com/plstcharles/thelper
pip3 install --quiet -e ./thelper

Cloning into 'thelper'...


We also install an external repo containing new models as well as useful functions.

In [2]:
!git clone https://github.com/sfoucher/gin-model-repo
!pip3 install --quiet -e ./gin-model-repo

Cloning into 'gin-model-repo'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 52 (delta 15), reused 38 (delta 11), pack-reused 0
Unpacking objects: 100% (52/52), done.


The first time you are installing thelper, you may need to restart the current session

In [ ]:
import os
os.kill(os.getpid(), 9)

Make sure thelper is imported correctly

In [1]:
import os
import torch
import torchvision
import thelper
import ginmodelrepo

thelper.utils.init_logger()

We download a ResNet-18 trained on DeepGlobe:

In [2]:
from ginmodelrepo.util import maybe_download_and_extract
maybe_download_and_extract('1Xl3LYBkmN-MVqBFR5eenHskOem-xVIY9','/content/model.pth')

Download finished. Extracting files.
Done.


Name your model

In [3]:
model_name = 'resnet18-deepglobe'

### Configuration

Specify your class names:

In [4]:
class_names = ["AgriculturalLand", "BarrenLand", "ForestLand", "RangeLand", "UrbanLand", "Water"]

#### Consulting the Taxonomy
You can consult the GeoImageNet Taxonomies, at level 0 it will you give how many taxonomies are available:

In [5]:
import requests
import json
response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?depth=0')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': []}, {'id': 1, 'name_fr': 'Objets', 'taxonomy_id': 1, 'code': 'OBJE', 'name_en': 'Objects', 'children': []}]


You can see that two taxonomies are available: Land cover and Objects. We can consult the Land Cover taxonomy at level 1

In [6]:
response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?taxonomy_name=land-cover&depth=1')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': [{'id': 206, 'name_fr': 'Urbain ou bâti', 'taxonomy_id': 2, 'code': 'URBA', 'name_en': 'Urban or Built-up Land', 'children': [{'id': 207, 'name_fr': 'Zone résidentielle', 'taxonomy_id': 2, 'code': 'RESD', 'name_en': 'Residential', 'children': [{'id': 208, 'name_fr': 'Densité faible', 'taxonomy_id': 2, 'code': 'DENS', 'name_en': 'Low density', 'children': []}, {'id': 209, 'name_fr': 'Densité moyenne', 'taxonomy_id': 2, 'code': 'DENI', 'name_en': 'Medium density', 'children': []}, {'id': 210, 'name_fr': 'Densité élevée', 'taxonomy_id': 2, 'code': 'DENT', 'name_en': 'High density', 'children': []}, {'id': 211, 'name_fr': 'Parc de maisons mobiles', 'taxonomy_id': 2, 'code': 'PARC', 'name_en': 'Mobile home park', 'children': []}]}, {'id': 212, 'name_fr': 'Zone commerciales et services', 'taxonomy_id': 2, 'code': 'COMC', 'name_en': 'Commercial and Services', 'children': [{'id':

Check that your classes are defined in the taxonomy

In [7]:

response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?taxonomy_name=land-cover')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': [{'id': 206, 'name_fr': 'Urbain ou bâti', 'taxonomy_id': 2, 'code': 'URBA', 'name_en': 'Urban or Built-up Land', 'children': [{'id': 207, 'name_fr': 'Zone résidentielle', 'taxonomy_id': 2, 'code': 'RESD', 'name_en': 'Residential', 'children': [{'id': 208, 'name_fr': 'Densité faible', 'taxonomy_id': 2, 'code': 'DENS', 'name_en': 'Low density', 'children': []}, {'id': 209, 'name_fr': 'Densité moyenne', 'taxonomy_id': 2, 'code': 'DENI', 'name_en': 'Medium density', 'children': []}, {'id': 210, 'name_fr': 'Densité élevée', 'taxonomy_id': 2, 'code': 'DENT', 'name_en': 'High density', 'children': []}, {'id': 211, 'name_fr': 'Parc de maisons mobiles', 'taxonomy_id': 2, 'code': 'PARC', 'name_en': 'Mobile home park', 'children': []}]}, {'id': 212, 'name_fr': 'Zone commerciales et services', 'taxonomy_id': 2, 'code': 'COMC', 'name_en': 'Commercial and Services', 'children': [{'id':

Define a class mapping between your class names and the taxonomy ids

In [8]:
class_mapping = [('AgriculturalLand', 223), ('BarrenLand', 252), ('ForestLand', 233), ('RangeLand', 229), ('UrbanLand', 201), ('Water', 239)]

#### Model Configuration

Define the task as a Classification task:

In [9]:
task_config = {
    "type": "thelper.tasks.Classification",
    "params": {
        "class_names": class_names,
        "input_key": "data",
        "label_key": "label"
    }
}
print(task_config)

{'type': 'thelper.tasks.Classification', 'params': {'class_names': ['AgriculturalLand', 'BarrenLand', 'ForestLand', 'RangeLand', 'UrbanLand', 'Water'], 'input_key': 'data', 'label_key': 'label'}}


We need to modify the task to include your class mapping

In [10]:
from ginmodelrepo.util import update_class_mapping
task = update_class_mapping(class_mapping, task_config)
print(task)

thelper.tasks.classif.Classification(class_names=['223', '252', '233', '229', '201', '239'], input_key='data', label_key='label', meta_keys=[], multi_label=False)


Define the data loader, you'll need to specify the bands you are using (starting with channel no 1):



In [11]:
channels = [1,2,3]
datasets_config = {
    "deepglobe_test": {
        "type": "thelper.data.geo.parsers.ImageFolderGDataset",
        "params": {"root": "/content/",
                    "image_key": "image",
                    "channels": channels

                    },
        "task": task
    }
}

Define the pre-processing pipeline as following:

In [12]:
loaders_config = {
    "base_transforms": [
        {
            "operation": "thelper.transforms.Resize",
            "params": {"dsize": [224, 224]},
        },
        {
            "operation": "thelper.transforms.NormalizeMinMax",
            "params": {
                "min": [0, 0, 0],
                "max": [255, 255, 255]
            },
        },
        {
            "operation": "thelper.transforms.NormalizeZeroMeanUnitVar",
            "params": {
                "mean": [0.485, 0.456, 0.406],
                "std": [0.229, 0.224, 0.225]
            },
        },
        {
            "operation": "torchvision.transforms.ToTensor",
        },
    ],
}

Define the model configuration:
* type: your model specific class
* params: the parameters needed by that class
* state_dict: points to the model parameters



In [13]:
model_config = {
    "type": torchvision.models.resnet18,
    "params": {"pretrained": True},
    'state_dict': '/content/model.pth',
    "task": task
}

### Export the model

Trained models are typically exported in two parts: architecture and weights. However, metadata related to the task the model was built for would be missing with only those two components. Here, we show a solution for exporting a classification model trained using the framework under ONNX, TraceScript, or pickle format along with its corresponding index-to-class-name mapping.

The advantage of ONNX and TraceScript exports is that whoever reloads your model does not need to have the class that you used to define the model's architecture at hand. 

Models/checkpoints exported this way can be easily reloaded using the framework, and may also be opened manually by others to extract only the information they need.

Define the thelper export parameters, make sure the trace_input corresponds to the input layer

In [25]:
tensor_sample =  torch.rand(1, 3, 224, 224)
export_config = {
    "ckpt_name": model_name + ".pth",
    "trace_name": model_name + ".zip",
    "save_raw": True,
    "trace_input": tensor_sample,
    "task": task
}

# Launch the export

If everything goes well,` /content/thelper-export/resnet18-deepglobe` should now contain a checkpoint with the exported model trace and all metadata required to reinstantiate it. Note that as of recently, PyTorch exports model traces as zip files, meaning you will have to copy two files from the output session folder. In this case, that would be resnet18-deepglobe.pth and resnet18-deepglobe.zip.

In [26]:
# config = {"name": model_name, "model": model_config, "datasets": datasets_config, "loaders": loaders_config, "export": export_config}
config = {"name": model_name, "model": model_config, "datasets": datasets_config, "loaders": loaders_config}
thelper.utils.bypass_queries = True     # avoid blocking ui query
thelper.cli.export_model(config, '/content/thelper-export')

[2020-09-08 15:42:18,716 - thelper.cli.export_model] INFO : exporting model 'resnet18-deepglobe'...
[2020-09-08 15:42:18,719 - thelper.utils.get_save_dir] INFO : output root directory = /content/thelper-export
[2020-09-08 15:42:18,972 - thelper.utils] DEBUG : bypassed query 'Training session at `/content/thelper-export/...' with True
[2020-09-08 15:42:18,973 - thelper.utils.get_save_dir] INFO : output session directory = /content/thelper-export/resnet18-deepglobe
[2020-09-08 15:42:18,974 - thelper.utils.get_save_dir] INFO : output logs directory = /content/thelper-export/resnet18-deepglobe/logs
[2020-09-08 15:42:18,990 - thelper.cli.export_model] DEBUG : exported checkpoint will be saved at '/content/thelper-export/resnet18-deepglobe'
[2020-09-08 15:42:18,992 - thelper.nn.utils] DEBUG : loading model
[2020-09-08 15:42:18,993 - thelper.nn.utils] DEBUG : loading model type/params current config
[2020-09-08 15:42:19,030 - thelper.nn.utils] DEBUG : model_type = <function resnet18 at 0x7f20

We can check that the export can be reloaded

In [22]:
from ginmodelrepo.util import load_model
model_file = '/content/thelper-export/resnet18-deepglobe/resnet18-deepglobe.export.pth'
success, model_checkpoint_info, model_buffer, excptions = load_model(model_file)
print(success)

[2020-09-08 15:05:03,756 - thelper.utils] DEBUG : testing availability of cuda device #0 (Tesla T4)


True


In [23]:
from ginmodelrepo.util import validate_model
success = validate_model(model_checkpoint_info)
print(success)

Model task not defined as dictionary nor `thelper.task.Task` class: [thelper.tasks.classif.Classification(class_names=['223', '252', '233', '229', '201', '239'], input_key='data', label_key='label', meta_keys=[], multi_label=False)]
Verifying model task as string: thelper.tasks.classif.Classification
Model task defined as string allowed after basic validation.
Forbidden model checkpoint task defined as string doesn't respect expected syntax.
False


Testing the TorchScript version of your model

In [ ]:
model = torch.jit.load('/content/thelper-export/deepglobe-unet/deepglobe-unet.zip')
model.eval()
print(model)
# print example output (should be same as during save)
x = torch.ones(1, 3, 128, 128)
print(model(x))

RecursiveScriptModule(
  original_name=ExternalModule
  (model): RecursiveScriptModule(
    original_name=EncoderDecoderNet
    (encoder1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Conv2d)
      (1): RecursiveScriptModule(original_name=BatchNorm2d)
      (2): RecursiveScriptModule(original_name=ReLU)
      (3): RecursiveScriptModule(original_name=MaxPool2d)
    )
    (encoder2): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      )
      (1): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=C

### Push your Model using GIN API

In [ ]:
To be finished...

In [24]:
!curl -X POST "https://geoimagenet.ca/ml/models" -H "accept: application/json" -H "Content-Type: application/json" -d "{ \"model_name\": \"{model_name}\", \"model_path\": \"{model_file}\"}"

<?xml version="1.0" encoding="utf-8"?>
<ExceptionReport version="1.0.0"
    xmlns="http://www.opengis.net/ows/1.1"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.opengis.net/ows/1.1 http://schemas.opengis.net/ows/1.1.0/owsExceptionReport.xsd">
    <Exception exceptionCode="NoApplicableCode" locator="AccessForbidden">
        <ExceptionText>Not authorized to access this resource. User does not meet required permissions.</ExceptionText>
    </Exception>
</ExceptionReport>